In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Imports
import os
import sklearn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall

In [ ]:
# Seed for reproducibility
tf.random.set_seed(4)

## Loading images

In [ ]:
tf.test.gpu_device_name()

In [ ]:
# Directory
dir_cur = os.getcwd()
print(dir_cur)    

In [ ]:
os.listdir()

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/plants_recognition')

In [ ]:
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
# Path to training data
path_training_data = Path("fruits/Training")

# Path to test data
path_test_data = Path("fruits/Test")

# Listing folder contents
training_img = list(path_training_data.glob("*/*"))

In [ ]:
# Lambda expression that extracts only the value with the path of each image
training_img = list(map(lambda x: str(x), training_img))

In [ ]:
# View a sample list
training_img[725:736]

In [ ]:
# Total training images
len(training_img)

## Data pre-processing

In [ ]:
# Function that gets the label of each image
def get_label(path_img):
    return path_img.split("/")[-2]

In [ ]:
# Apply the function
img_training_labels = list(map(lambda x: get_label(x), training_img))

In [ ]:
# Visualizing sample
img_training_labels[740:751]

In [ ]:
# Create the object
encoder = LabelEncoder()

In [ ]:
# Apply the fit_transform
img_training_labels = encoder.fit_transform(img_training_labels)

In [ ]:
# Visualizing sample
img_training_labels[740:745]

In [ ]:
# Apply One-Hot-Encoding on the labels
img_training_labels = tf.keras.utils.to_categorical(img_training_labels)

In [ ]:
# Visualizing sample
img_training_labels[740:745]

In [ ]:
# Split the training data into two samples, training and validation
X_traning, X_valid, Y_traning, Y_valid = train_test_split(training_img, img_training_labels)

In [ ]:
print("X Training\n", X_traning[10:13])
print("Y Training\n",Y_traning[10:13])

## Dataset Augmentation

In [ ]:
# Resizing all images to 224 x 224
img_size = 224
resize = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Resizing(img_size, img_size)])

# Create object for augmentation dataset
data_augmentation = tf.keras.Sequential([RandomFlip("horizontal"),
                                         RandomRotation(0.2),
                                         RandomZoom(height_factor = (-0.3,-0.2)) ])

## Data preparation

In [ ]:
# Hyperparameters
batch_size = 32
autotune = tf.data.experimental.AUTOTUNE

In [ ]:
# Function to load and transform images
def loading_transform(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    return image, label

In [ ]:
# Function to prepare data in TensorFlow format
def prepare_dataset(path, labels, train = True):

    # Prepare the data
    image_paths = tf.convert_to_tensor(path)
    labels = tf.convert_to_tensor(labels)
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    dataset = dataset.map(lambda image, label: loading_transform(image, label)) 
    dataset = dataset.map(lambda image, label: (resize(image), label), num_parallel_calls = autotune)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)

    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label), num_parallel_calls = autotune)
  
    dataset = dataset.repeat()

    return dataset

In [ ]:
# Create the training dataset
dataset_training = prepare_dataset(X_traning, Y_traning)

In [ ]:
# Shape
img, label = next(iter(dataset_training))
print(img.shape)
print(label.shape)

In [ ]:
# Visualize an image and a label
print(encoder.inverse_transform(np.argmax(label, axis = 1))[0])
plt.imshow((img[0].numpy()/255).reshape(224,224,3))

In [ ]:
# Create the validation dataset
dataset_valid = prepare_dataset(X_valid, Y_valid, train = False)

In [ ]:
# Shape
img, label = next(iter(dataset_valid))
print(img.shape) 
print(label.shape)

## Building the model

In [ ]:
# Loading a pre-trained model
model_pre = EfficientNetB3(input_shape = (224,224,3), include_top = False)

In [ ]:
# Adding our own layers to model_pre
model = tf.keras.Sequential([model_pre,
                              tf.keras.layers.GlobalAveragePooling2D(),
                              tf.keras.layers.Dense(131, activation = 'softmax')])

In [ ]:
# Model summary
model.summary()

In [ ]:
# Hyperparameters
lr = 0.001
beta1 = 0.9
beta2 = 0.999
ep = 1e-07

In [ ]:
# Compile model
model.compile(optimizer = Adam(learning_rate = lr, 
                                beta_1 = beta1, 
                                beta_2 = beta2, 
                                epsilon = ep),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy', Precision(name = 'precision'), Recall(name = 'recall')])

In [ ]:
%%time
history = model.fit(dataset_training,
                     steps_per_epoch = len(X_traning)//batch_size,
                     epochs = 1,
                     validation_data = dataset_valid,
                     validation_steps = len(Y_traning)//batch_size)

In [ ]:
# Don't need the model_pre anymore
model.layers[0].trainable = False

In [ ]:
# Checkpoint
bestModel = tf.keras.callbacks.ModelCheckpoint("model/best_model.h5", 
                                                verbose = 1, 
                                                save_best = True, 
                                                save_weights_only = True)

In [ ]:
# Early stop
early_stop = tf.keras.callbacks.EarlyStopping(patience = 4) 

In [ ]:
# Summary
model.summary()

In [ ]:
%%time
history = model.fit(dataset_training,
                     steps_per_epoch = len(X_traning)//batch_size,
                     epochs = 6,
                     validation_data = dataset_valid,
                     validation_steps = len(Y_traning)//batch_size,
                     callbacks = [bestModel, early_stop])

## Model evaluation

In [ ]:
# To load the weights we need to unfreeze the layers
model.layers[0].trainable = True

In [ ]:
# Load checkpoint weights and re-evaluate
model.load_weights("model/best_model.h5")

In [ ]:
# Loading and preparing test data
path_img_test = list(path_test_data.glob("*/*"))
test_img = list(map(lambda x: str(x), path_img_test))
img_test_labels = list(map(lambda x: get_label(x), test_img))
img_test_labels = encoder.fit_transform(img_test_labels)
img_test_labels = tf.keras.utils.to_categorical(img_test_labels)
test_image_paths = tf.convert_to_tensor(test_img)
img_test_labels = tf.convert_to_tensor(img_test_labels)

In [ ]:
# Image decode function
def decode_img(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [224,224], method = "bilinear")
    return image, label

In [ ]:
# Create the test dataset
dataset_test = (tf.data.Dataset
                 .from_tensor_slices((test_img, img_test_labels))
                 .map(decode_img)
                 .batch(batch_size))

In [ ]:
# Shape
img, label = next(iter(dataset_test))
print(img.shape)
print(label.shape)

In [ ]:
# Visualizing a test image
print(encoder.inverse_transform(np.argmax(label, axis = 1))[0])
plt.imshow((img[0].numpy()/255).reshape(224,224,3))

In [ ]:
# Evaluating the model
loss, acc, prec, rec = model.evaluate(dataset_test)

In [ ]:
print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)

## Trained Model Predicting

In [ ]:
# Function to load a new image
def loading_new_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, [224,224], method = "bilinear")
    plt.imshow(img.numpy()/255)
    img = tf.expand_dims(img, 0) 
    return img

In [ ]:
# Function to make predictions
def predition_func(img_path, model, enc):
    img = loading_new_img(img_path)
    prediction = model.predict(img)
    pred = np.argmax(prediction, axis = 1) 
    return enc.inverse_transform(pred)[0] 

In [ ]:
# Prediction
predition_func("images/image1.jpg", model, encoder)

In [ ]:
# Prediction
predition_func("images/image2.jpg", model, encoder)

In [ ]:
# Prediction
predition_func("images/image3.jpg", model, encoder)

In [ ]:
# Prediction
predition_func("images/image4.jpg", model, encoder)

In [ ]:
# Prediction
predition_func("images/image5.jpg", model, encoder)